# Анализ каналов привлечения

# Описание проекта

В ходе проекта необходимо выполнить: 
- Расчет CPI и ROAS для двух маркетинговых каналов привлечения клиентов.
- Расчитать конверсию в платящего пользователя для канала MediaSource2.
- Сформулировать рекомендации для оптимизации рекламных компаний.
- Описать методы и модели атрибуции.
- Описать решение кейса с игрой.
- Составить топ мобильных игр и описать их механику.



# 1. Изучение общей информации

## 1.1 Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

## 1.2 Чтение файлов

In [2]:
try:
    installs = pd.read_csv('/Users/angelinazigalova/Desktop/dataset - Installs.csv')
    payments = pd.read_csv('/Users/angelinazigalova/Desktop/dataset - Payments.csv')
    cost_v1 = pd.read_csv('/Users/angelinazigalova/Desktop/dataset - Cost_v1.csv')
    cost_v2 = pd.read_csv('/Users/angelinazigalova/Desktop/dataset - Cost_v2.csv')
except:
    print('Не удалось открыть файлы')

## 1.3 Просмотр общей информации о данных

In [3]:
installs.head(5)

,MediaSource,CampaignID,Channel,Country,Wifi,UserID,OSVersion
0,MediaSource1,19560305480,Search,AR,True,1673554038946-5942926321601974416,8
1,MediaSource1,19560308588,Network,ZA,True,1673647857853-6408906530696604886,8
2,MediaSource1,19560305480,Search,HN,False,1673645658337-5378307560700406034,8
3,MediaSource1,19560308588,Network,FR,True,1673642511260-6240081802543562050,8
4,MediaSource1,19560308588,Network,ES,True,1673641432689-6862189683515188142,8


In [4]:
installs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22416 entries, 0 to 22415
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MediaSource  22416 non-null  object
 1   CampaignID   22416 non-null  object
 2   Channel      22416 non-null  object
 3   Country      22409 non-null  object
 4   Wifi         22416 non-null  bool  
 5   UserID       22416 non-null  object
 6   OSVersion    22416 non-null  int64 
dtypes: bool(1), int64(1), object(5)
memory usage: 1.0+ MB


In [5]:
payments.head(5)

,UserID,PaymentRevenue
0,1673672646759-6010710452938637680,"5,826125793"
1,1673811886533-3308641358962027481,"1,163524761"
2,1673557631838-4075714709968288531,"6,056467465"
3,1673557631838-4075714709968288531,"6,056467465"
4,1673713438526-5393379852337099847,"3,832763742"


In [6]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1580 entries, 0 to 1579
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   UserID          1580 non-null   object
 1   PaymentRevenue  1580 non-null   object
dtypes: object(2)
memory usage: 24.8+ KB


In [7]:
cost_v1.head(5)

,MediaSource,CampaignID,Country,Cost
0,MediaSource1,19560305480,UG,"0,628868"
1,MediaSource1,19560305480,MW,"0,253214"
2,MediaSource1,19560308588,NaN,"0,84028"
3,MediaSource1,19560305480,LR,"0,046751"
4,MediaSource1,19560308588,LK,"0,858504"


In [8]:
cost_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MediaSource  435 non-null    object
 1   CampaignID   435 non-null    object
 2   Country      431 non-null    object
 3   Cost         435 non-null    object
dtypes: object(4)
memory usage: 13.7+ KB


In [9]:
cost_v2.head(5)

,MediaSource,CampaignID,Channel,Cost
0,MediaSource1,19560308588,Network,"2594,5129"
1,MediaSource1,19560305480,Network,"2661,085983"
2,MediaSource1,19560308588,Search,"1206,427759"
3,MediaSource1,19560305480,Search,"1105,145814"
4,MediaSource1,19560305480,VideoNetwork,"12,431458"


In [10]:
cost_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MediaSource  24 non-null     object
 1   CampaignID   24 non-null     object
 2   Channel      24 non-null     object
 3   Cost         24 non-null     object
dtypes: object(4)
memory usage: 896.0+ bytes


В таблице installs есть пропуски в столбце со страной, их можно заполнить

Неправильный тип столбца таблицы payments - PaymentRevenue

В таблице cost_v1 есть пропуски в встолбце страны и не верный тип столбца Cost

В таблице cost_v2 неправильный тип стообца Cost

Также нужно привести названия столбцов к удобному виду

# 2. Предобработка данных

## 2.1 Поиск явных дубликатов

In [11]:
print('Количество найденых дубликатов:', installs.duplicated().sum())
print('Количество найденых дубликатов:', payments.duplicated().sum())
print('Количество найденых дубликатов:', cost_v1.duplicated().sum())
print('Количество найденых дубликатов:', cost_v2.duplicated().sum())

Количество найденых дубликатов: 1
Количество найденых дубликатов: 486
Количество найденых дубликатов: 0
Количество найденых дубликатов: 0


## 2.2 Привожу столбцы к единому виду

In [12]:
installs.columns = ['media_source', 'campaign_id', 'channel', 'country', 'wifi', 'user_id', 'os_version']
payments.columns = ['user_id', 'payment_revenue']
cost_v1.columns = ['media_source', 'campaign_id', 'country', 'cost']
cost_v2.columns = ['media_source', 'campaign_id', 'channel', 'cost']

## 2.3  Привожу столбцы к нужному типу

In [13]:
cost_v1['cost'] = cost_v1['cost'].str.replace(',', '.').astype(float)
cost_v2['cost'] = cost_v2['cost'].str.replace(',', '.').astype(float)
payments['payment_revenue'] = payments['payment_revenue'].str.replace(',', '.').astype(float)

# 3. Расчет CPI и ROAS

Cost Per Install рассчитываю как затраты на рекламу/количество установок

In [14]:
# Расчет затрат на рекламу для каждого источника трафика
total_costs = cost_v1.pivot_table(index = 'media_source', values = 'cost', aggfunc = 'sum').reset_index()
total_costs.columns = ['media_source', 'total_cost']
total_costs['total_cost'] = round(total_costs['total_cost'], 2)
total_costs

,media_source,total_cost
0,MediaSource1,7665.15
1,MediaSource2,12260.20


In [15]:
# Общее количество установок для каждого источника трафика
total_installs = installs.pivot_table(index = 'media_source', values = 'user_id', aggfunc = 'count').reset_index()
total_installs.columns = ['media_source', 'installs']
total_installs

,media_source,installs
0,MediaSource1,19717
1,MediaSource2,2699


In [16]:
# Расчет cpi
cpi = pd.merge(total_costs, total_installs, on='media_source', how='inner')
cpi['cpi'] = round(cpi['total_cost'] / cpi['installs'], 2)
cpi

,media_source,total_cost,installs,cpi
0,MediaSource1,7665.15,19717,0.39
1,MediaSource2,12260.20,2699,4.54


Рентабельность рекламных расходов считаю как общий доход/общие затраты на рекламу

In [17]:
# Считаю суммарный доход на каждого пользователя
pay_per_user = payments.pivot_table(index = 'user_id', values = 'payment_revenue', aggfunc = 'sum').reset_index()
# Для каждого пользователя определяю канал привлечения
media_per_user = installs.pivot_table(index = 'user_id', values = 'media_source', aggfunc = 'first').reset_index()
# Объединяю таблицы
total_revenue = pd.merge(media_per_user, pay_per_user, on='user_id', how='inner')
# Считаю доход для каждого канала привлечения
revenue = total_revenue.pivot_table(index = 'media_source', values = 'payment_revenue', aggfunc = 'sum').reset_index()
revenue.columns = ['media_source', 'revenue']
roas = pd.merge(cpi, revenue, on='media_source', how='inner')
roas['revenue'] = round(roas['revenue'], 2)
roas

,media_source,total_cost,installs,cpi,revenue
0,MediaSource1,7665.15,19717,0.39,4977.34
1,MediaSource2,12260.20,2699,4.54,6857.23


In [18]:
# Считаю ROAS
roas['roas'] = round((roas['revenue']/roas['total_cost']) * 100, 2)
roas

,media_source,total_cost,installs,cpi,revenue,roas
0,MediaSource1,7665.15,19717,0.39,4977.34,64.93
1,MediaSource2,12260.20,2699,4.54,6857.23,55.93


Стоимость одной установки по каналу MediaSource1 - 39 цента, что в 11 раз ниже, чем инстановка по каналу MediaSource2.

Окупаемость рекламы в MediaSource1 выше - почти 65%. Отрыв в 9% от MediaSource2, что достаточно значимо.

# 4. Расчет конверсии в платящего пользователя для канала MediaSource2

In [19]:
# Фильтрую данные по каналу привлечения
data = installs.query('media_source == "MediaSource2"')
# Считаю установки в разбивке по wifi
data = data.pivot_table(index = 'wifi', values = 'user_id', aggfunc = 'count').reset_index()
data.columns = ['wifi', 'total_installs']
data

,wifi,total_installs
0,False,988
1,True,1711


In [20]:
# Считаю покупки в разбивке по  wifi
payment_group = payments.pivot_table(index = 'user_id', values = 'payment_revenue', aggfunc = 'sum').reset_index()
installs_group = installs.pivot_table(index = 'user_id', values = 'wifi', aggfunc = 'first').reset_index()
payer = pd.merge(installs_group, payment_group, on='user_id', how='left')
payer['payment_revenue'] = payer['payment_revenue'].fillna(0)
payer = payer.query('payment_revenue != 0.0')
payer = payer.pivot_table(index = 'wifi', values = 'payment_revenue', aggfunc = 'count').reset_index()
payer.columns = ['wifi', 'total_payer']
payer

,wifi,total_payer
0,False,172
1,True,335


In [21]:
# Считаю конверсию
conversion = pd.merge(data, payer, on = 'wifi', how = 'inner')
conversion['conversion'] = round(conversion['total_payer']/conversion['total_installs'], 2)
conversion

,wifi,total_installs,total_payer,conversion
0,False,988,172,0.17
1,True,1711,335,0.20


Судя по конверсии - наличие  wifi  немного влияет на покупки.

Для того, чтобы утверждение было с большой вероятность верным, необходимо провевести статистический тест на различие долей.

Сформулирую нулевую и альтернативну гипотезы:
- Н0: Доли платящих пользователей с wifi и без него равны
- Н1: Доли не равны

In [22]:
# Возьму значения количества установок и покупок в каждой группе
wifi_installs = conversion.loc[1, 'total_installs']
no_wifi_installs = conversion.loc[0, 'total_installs']
wifi_payer = conversion.loc[1, 'total_payer']
no_wifi_payer = conversion.loc[0, 'total_payer']
# провожу тест
count = np.array([wifi_payer, no_wifi_payer])
nobs = np.array([wifi_installs, no_wifi_installs])
alpha = 0.05
z_stat, p_value = proportions_ztest(count, nobs)

if p_value < alpha:
    print("Можно отвергнуть нулевую гипотезу - доли не равны.", f"Z-статистика: {z_stat}", f"P-значение: {p_value}")
else:
    print("Не удалось отвергнуть гипотезу - доли равны.", f"Z-статистика: {z_stat}", f"P-значение: {p_value}")

Не удалось отвергнуть гипотезу - доли равны. Z-статистика: 1.390585664759975 P-значение: 0.16435110583632384


Z-статистика значительно отличается от нуля, а Р-значение меньше уровня значимости 0.05 - значит я могу утверждать, что платящих пользователей с  wifi и без него равны, что говорит, о том, что показатель конверсии показывает действительную картину - пользователи с wifi чаще совершают покупку в приложении.

# 5. Рекомендации для оптимизации рекламных компаний

В первую очередь я бы перераспределил бюджет в пользу каналов, показывающих больший потенциал по ROAS - в нашем случае MediaSource1. Этоувеличит прибыль в краткосрочной перспективе. 

Далее можно точечно работать с отдельными показателями каналов привлечения и их количеством.

Например, стоимость установки приложения для канала MediaSource2 в 11 раз больше, чем для MediaSource1. Можно поробовать снизить стоимость.


## 5.1 MediaSource1

In [23]:
df = installs.query('media_source == "MediaSource1"')
df.pivot_table(index = 'country', values = 'user_id', aggfunc = 'count').reset_index().sort_values(by = 'user_id', ascending = False).head(10)

,country,user_id
22,BR,3612
6,AR,1304
122,MX,1034
169,TR,880
181,VN,662
58,FR,591
33,CL,585
137,PL,574
79,ID,563
52,ES,536


MediaSource1 настроен на рекламу в разных странах

Попробую посмотреть на показатели для MediaSource1 в разбивке по сранам

In [24]:
total_costs =  cost_v1.query('media_source == "MediaSource1"')
total_costs = total_costs.pivot_table(index = 'country', values = 'cost', aggfunc = 'sum').reset_index()
total_costs.columns = ['country', 'total_cost']
total_costs['total_cost'] = round(total_costs['total_cost'], 2)
total_installs =  installs.query('media_source == "MediaSource1"')
total_installs = total_installs.pivot_table(index = 'country', values = 'user_id', aggfunc = 'count').reset_index()
total_installs.columns = ['country', 'installs']
cpi = pd.merge(total_costs, total_installs, on='country', how='inner')
cpi['cpi'] = round(cpi['total_cost'] / cpi['installs'], 2)
pay_per_user = payments.pivot_table(index = 'user_id', values = 'payment_revenue', aggfunc = 'sum').reset_index()
country_per_user = installs.pivot_table(index = 'user_id', values = 'country', aggfunc = 'first').reset_index()
total_revenue = pd.merge(country_per_user, pay_per_user, on='user_id', how='inner')
revenue = total_revenue.pivot_table(index = 'country', values = 'payment_revenue', aggfunc = 'sum').reset_index()
revenue.columns = ['country', 'revenue']
roas = pd.merge(cpi, revenue, on='country', how='inner')
roas['revenue'] = round(roas['revenue'], 2)
roas['roas'] = round((roas['revenue']/roas['total_cost']) * 100, 2)
roas.sort_values(by = ['installs', 'roas'], ascending = False).head(17)

,country,total_cost,installs,cpi,revenue,roas
7,BR,711.88,3612,0.20,529.15,74.33
1,AR,139.52,1304,0.11,62.01,44.45
38,MX,212.01,1034,0.21,311.07,146.72
57,TR,232.41,880,0.26,97.18,41.81
62,VN,167.70,662,0.25,98.57,58.78
22,FR,560.03,591,0.95,315.50,56.34
9,CL,170.50,585,0.29,262.11,153.73
48,PL,234.40,574,0.41,319.74,136.41
27,ID,168.29,563,0.30,29.51,17.54
21,ES,253.36,536,0.47,74.28,29.32


Средний показатель roas для канала - 65.

Можно выделить Бразилию, Мексику, Чили и Польшу - оккупаемость в этих сранах превышает среднюю по каналу, значит бюджет рекламных расходов для этих стран можно увеличить.

Интересно, что реклама идет и на рынок США, причем с достаточно дешевой установкой, что показывает огромную окупаемость. Правда, это может быть связано с использованием  VPN.

Посмотрим на показатели каналов

In [25]:
total_costs =  cost_v2.query('media_source == "MediaSource1"')
total_costs = total_costs.pivot_table(index = 'channel', values = 'cost', aggfunc = 'sum').reset_index()
total_costs.columns = ['channel', 'total_cost']
total_costs['total_cost'] = round(total_costs['total_cost'], 2)
total_installs =  installs.query('media_source == "MediaSource1"')
total_installs = total_installs.pivot_table(index = 'channel', values = 'user_id', aggfunc = 'count').reset_index()
total_installs.columns = ['channel', 'installs']
cpi = pd.merge(total_costs, total_installs, on='channel', how='inner')
cpi['cpi'] = round(cpi['total_cost'] / cpi['installs'], 2)
pay_per_user = payments.pivot_table(index = 'user_id', values = 'payment_revenue', aggfunc = 'sum').reset_index()
channel_per_user = installs.pivot_table(index = 'user_id', values = 'channel', aggfunc = 'first').reset_index()
total_revenue = pd.merge(channel_per_user, pay_per_user, on='user_id', how='inner')
revenue = total_revenue.pivot_table(index = 'channel', values = 'payment_revenue', aggfunc = 'sum').reset_index()
revenue.columns = ['channel', 'revenue']
roas = pd.merge(cpi, revenue, on='channel', how='inner')
roas['revenue'] = round(roas['revenue'], 2)
roas['roas'] = round((roas['revenue']/roas['total_cost']) * 100, 2)
roas

,channel,total_cost,installs,cpi,revenue,roas
0,Network,5255.60,13324,0.39,3538.23,67.32
1,Search,2311.57,6342,0.36,1428.60,61.80
2,VideoNetwork,99.03,51,1.94,10.52,10.62


Бюджет расходуется на самый выгодный канал Network

## 5.2 MediaSource2

In [26]:
dff = installs.query('media_source == "MediaSource2"')
dff.pivot_table(index = 'country', values = 'user_id', aggfunc = 'count').reset_index().sort_values(by = 'user_id', ascending = False).head(5)

,country,user_id
8,US,2687
3,MX,3
1,CA,2
0,AR,1
2,CO,1


MediaSource2 фактичеки настроен только на США. Это объясняет дороговизну установки.

Попробую посмотреть на каналы

In [27]:
total_costs =  cost_v2.query('media_source == "MediaSource2"')
total_costs = total_costs.pivot_table(index = 'channel', values = 'cost', aggfunc = 'sum').reset_index()
total_costs.columns = ['channel', 'total_cost']
total_costs['total_cost'] = round(total_costs['total_cost'], 2)
total_installs =  installs.query('media_source == "MediaSource2"')
total_installs = total_installs.pivot_table(index = 'channel', values = 'user_id', aggfunc = 'count').reset_index()
total_installs.columns = ['channel', 'installs']
cpi = pd.merge(total_costs, total_installs, on='channel', how='inner')
cpi['cpi'] = round(cpi['total_cost'] / cpi['installs'], 2)
pay_per_user = payments.pivot_table(index = 'user_id', values = 'payment_revenue', aggfunc = 'sum').reset_index()
channel_per_user = installs.pivot_table(index = 'user_id', values = 'channel', aggfunc = 'first').reset_index()
total_revenue = pd.merge(channel_per_user, pay_per_user, on='user_id', how='inner')
revenue = total_revenue.pivot_table(index = 'channel', values = 'payment_revenue', aggfunc = 'sum').reset_index()
revenue.columns = ['channel', 'revenue']
roas = pd.merge(cpi, revenue, on='channel', how='inner')
roas['revenue'] = round(roas['revenue'], 2)
roas['roas'] = round((roas['revenue']/roas['total_cost']) * 100, 2)
roas.sort_values(by = 'total_cost', ascending = False)

,channel,total_cost,installs,cpi,revenue,roas
8,IRONSOURCE,3789.68,912,4.16,1536.54,40.55
2,APPLOVIN,2351.20,567,4.15,1147.90,48.82
12,UNITY,2238.40,503,4.45,551.08,24.62
6,FYBER,1022.04,202,5.06,92.81,9.08
7,INMOBI,622.11,116,5.36,404.18,64.97
1,ADX,504.71,77,6.55,67.88,13.45
0,ADCOLONY,449.97,58,7.76,65.85,14.63
13,VUNGLE,388.37,75,5.18,373.40,96.15
11,TAPJOY,337.86,52,6.50,44.89,13.29
5,CHARTBOOST,241.75,27,8.95,33.96,14.05


В среднем оккупаемость ремкламы для MediaSource2 - 56.

Рекламные платформы INMOBI и VUNGLE показывают хороший результат оккупаемости рекламы. Стоит перераспределить финансирование в их пользу.

# 6. Методы и модели атрибуции

1. IDFA
Используется для отслеживания пользователя в мобильной рекламе, позволяет отслеживать, откуда пришли пользователи и какие источники рекламы были эффективны.

Из преимуществ можно выделить точное отслеживание пользователей и их взаимодейсвие с рекламой.

2. SDK

Позволяет получать более детальные данные - события в приложении, конверсии и другие метрики.

Из преимуществ можно выделить наличи дополнительнх данных и гибкость в настройке.

# 7. Игра

Можно попробовать метод Монте-Карло и в ходе симуляции посмотреть на вероятность прохождения каждого уровня.

# 8. Топ мобильных игр

1. Mushroom Wars 2. PVP игра, где игроку необходимо захватывать здания противника - руководить своей экономикой и моралью войск. Все игроки начинают игру с одинаковым положением и побеждает тот, кто предугадает действия противника и сможет завоевать все его постройки. Можно играть 1 на 1, втроем или вчетвером каждый сам за себя, или командами 2 на 2.
2. Rebel Inc. Стратегическая игра за губернатора региона. Игроку выделяется бюджет и он должен его верно распределить между инициативами для мирного населения, инфраструкторой и войной с мятежниками. Необходимо полность стабилизировать регион. В игре предполагаются разные губернаторы и помощники, дающие разные бонусы.
3. Plague Inc. Игра от того же разработчика с такой же механикой. Тут необходимо изобрести лекарство от болезни. Ты также распоряжаешься своим бюджетом и делаешь приоритетный выбор.
4. Wild Rift. MOBA игра, где игроки играют в команде 5 на 5 и должны разрушить трон противника. У каждого игрока своя роль - помощь, поглащение урона или атака. Победа зависит от заработанного золота за убийство крипов или героев и купленных предметов.